In [1]:
import ghmm
from ghmm import *
import numpy as np

f = lambda x: "%.8f" % (x,) # float rounding function

def get_model_parameters(hmm_model, A_star, B_star, pi_star):
        hmm = hmm_model.cmodel

        if hmm_model.hasFlags(kHigherOrderEmissions):
            order = ghmmwrapper.int_array2list(hmm_model.cmodel.order, hmm_model.N)
        else:
            order = [0]*hmm.N

        if hmm.N <= 4:
            iter_list = range(hmm_model.N)
        else:
            iter_list = [0,1,'X',hmm.N-2,hmm.N-1]

        for k in iter_list:
            if k == 'X':
                continue

            state = hmm.getState(k)

            pi_star[k] = state.pi

            for outp in range(hmm.M**(order[k]+1)):
                B_star[k][outp] = ghmmwrapper.double_array_getitem(state.b,outp)

            for i in range( state.out_states):
                A_star[k][state.getOutState(i)] = ghmmwrapper.double_array_getitem(state.out_a,i)


def print_pi(pi, bw_str):
    for i in range(pi.shape[0]):
        print("    {}.init_prob[{}] = ".format(bw_str, i) + f(pi[i]) + ";")


def print_A(A, bw_str):
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            print("    {}.trans_prob[{}*N + {}] = ".format(bw_str, i, j) + f(A[i][j]) + ";")


def print_B(B, bw_str):
    for i in range(B.shape[0]):
        for j in range(B.shape[1]):
            print("    {}.emit_prob[{}*M + {}] = ".format(bw_str, i, j) + f(B[i][j]) + ";")


def print_model(A, B, pi, bw_str):
    print_pi(pi, bw_str)
    print("")
    print_A(A, bw_str)
    print("")
    print_B(B, bw_str)
    print("")


def print_converged_model(hmm_model, A, B, pi, bw_str):
    A_star = np.copy(A)
    B_star = np.copy(B)
    pi_star = np.copy(pi)
    get_model_parameters(hmm_model, A_star, B_star, pi_star)
    print_model(A_star, B_star, pi_star, bw_str)

def print_observations(Y, bw_str):
    print("")
    for i in range(Y.shape[0]):
        print("    {}.observations[{}*T + {}] = ".format(bw_str, 0, i) + str(Y[i]) + ";")
    print("")





In [2]:
# test_case_ghmm_0

A = np.array([[0.5, 0.5], [0.3, 0.7]], dtype=np.float64)
B = np.array([[0.3, 0.7], [0.8, 0.2]], dtype=np.float64)
pi = np.array([0.2, 0.8], dtype=np.float64)
Y = np.array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0], np.int)

nrSteps = 1
loglikelihoodCutoff = 0
sigma = IntegerRange(0,2)

print_observations(Y, "bw")
print_model(A, B, pi, "bw")
hmm_model = HMMFromMatrices(sigma, DiscreteDistribution(sigma), list(A), list(B), list(pi))
train_seq = EmissionSequence(sigma, list(Y))
hmm_model.baumWelch(train_seq, nrSteps, loglikelihoodCutoff)
print_converged_model(hmm_model, A, B, pi, "bw_check")




    bw.observations[0*T + 0] = 0;
    bw.observations[0*T + 1] = 0;
    bw.observations[0*T + 2] = 0;
    bw.observations[0*T + 3] = 0;
    bw.observations[0*T + 4] = 0;
    bw.observations[0*T + 5] = 1;
    bw.observations[0*T + 6] = 1;
    bw.observations[0*T + 7] = 0;
    bw.observations[0*T + 8] = 0;
    bw.observations[0*T + 9] = 0;

    bw.init_prob[0] = 0.20000000;
    bw.init_prob[1] = 0.80000000;

    bw.trans_prob[0*N + 0] = 0.50000000;
    bw.trans_prob[0*N + 1] = 0.50000000;
    bw.trans_prob[1*N + 0] = 0.30000000;
    bw.trans_prob[1*N + 1] = 0.70000000;

    bw.emit_prob[0*M + 0] = 0.30000000;
    bw.emit_prob[0*M + 1] = 0.70000000;
    bw.emit_prob[1*M + 0] = 0.80000000;
    bw.emit_prob[1*M + 1] = 0.20000000;

    bw_check.init_prob[0] = 0.07187023;
    bw_check.init_prob[1] = 0.92812977;

    bw_check.trans_prob[0*N + 0] = 0.43921478;
    bw_check.trans_prob[0*N + 1] = 0.56078522;
    bw_check.trans_prob[1*N + 0] = 0.21445682;
    bw_check.trans_prob[1*N + 1] = 0.7855

In [3]:
# test_case_ghmm_1
# the same as test_case_ghmm_0
# run until convergence, instead of just 1 iteration

A = np.array([[0.5, 0.5], [0.3, 0.7]], dtype=np.float64)
B = np.array([[0.3, 0.7], [0.8, 0.2]], dtype=np.float64)
pi = np.array([0.2, 0.8], dtype=np.float64)
Y = np.array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0], np.int)

nrSteps = 1000
loglikelihoodCutoff = 0
sigma = IntegerRange(0,2)

print_observations(Y, "bw")
print_model(A, B, pi, "bw")
hmm_model = HMMFromMatrices(sigma, DiscreteDistribution(sigma), list(A), list(B), list(pi))
train_seq = EmissionSequence(sigma, list(Y))
hmm_model.baumWelch(train_seq, nrSteps, loglikelihoodCutoff)
print_converged_model(hmm_model, A, B, pi, "bw_check")



GHMM ghmm.py:148 - reestimate.c:ghmm_dmodel_baum_welch(843): No convergence: log P < log P-old! (n=128)


    bw.observations[0*T + 0] = 0;
    bw.observations[0*T + 1] = 0;
    bw.observations[0*T + 2] = 0;
    bw.observations[0*T + 3] = 0;
    bw.observations[0*T + 4] = 0;
    bw.observations[0*T + 5] = 1;
    bw.observations[0*T + 6] = 1;
    bw.observations[0*T + 7] = 0;
    bw.observations[0*T + 8] = 0;
    bw.observations[0*T + 9] = 0;

    bw.init_prob[0] = 0.20000000;
    bw.init_prob[1] = 0.80000000;

    bw.trans_prob[0*N + 0] = 0.50000000;
    bw.trans_prob[0*N + 1] = 0.50000000;
    bw.trans_prob[1*N + 0] = 0.30000000;
    bw.trans_prob[1*N + 1] = 0.70000000;

    bw.emit_prob[0*M + 0] = 0.30000000;
    bw.emit_prob[0*M + 1] = 0.70000000;
    bw.emit_prob[1*M + 0] = 0.80000000;
    bw.emit_prob[1*M + 1] = 0.20000000;

    bw_check.init_prob[0] = 0.00000000;
    bw_check.init_prob[1] = 1.00000000;

    bw_check.trans_prob[0*N + 0] = 0.50000000;
    bw_check.trans_prob[0*N + 

In [4]:
# test_case_ghmm_2

A = np.array([[0.5, 0.4, 0.1], [0.3, 0.3, 0.4], [0.1, 0.1, 0.8]], dtype=np.float64)
B = np.array([[0.1, 0.0, 0.9], [0.05, 0.95, 0.0], [0.3, 0.52, 0.18]], dtype=np.float64)
pi = np.array([0.1, 0.5, 0.4], dtype=np.float64)
Y = np.array([2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 2, 2, 0, 1, 1, 0], np.int)

nrSteps = 1
loglikelihoodCutoff = 0
sigma = IntegerRange(0,3)

print_observations(Y, "bw")
print_model(A, B, pi, "bw")
hmm_model = HMMFromMatrices(sigma, DiscreteDistribution(sigma), list(A), list(B), list(pi))
train_seq = EmissionSequence(sigma, list(Y))
hmm_model.baumWelch(train_seq, nrSteps, loglikelihoodCutoff)
print_converged_model(hmm_model, A, B, pi, "bw_check")




    bw.observations[0*T + 0] = 2;
    bw.observations[0*T + 1] = 1;
    bw.observations[0*T + 2] = 0;
    bw.observations[0*T + 3] = 1;
    bw.observations[0*T + 4] = 0;
    bw.observations[0*T + 5] = 2;
    bw.observations[0*T + 6] = 2;
    bw.observations[0*T + 7] = 2;
    bw.observations[0*T + 8] = 1;
    bw.observations[0*T + 9] = 1;
    bw.observations[0*T + 10] = 2;
    bw.observations[0*T + 11] = 2;
    bw.observations[0*T + 12] = 0;
    bw.observations[0*T + 13] = 1;
    bw.observations[0*T + 14] = 1;
    bw.observations[0*T + 15] = 0;

    bw.init_prob[0] = 0.10000000;
    bw.init_prob[1] = 0.50000000;
    bw.init_prob[2] = 0.40000000;

    bw.trans_prob[0*N + 0] = 0.50000000;
    bw.trans_prob[0*N + 1] = 0.40000000;
    bw.trans_prob[0*N + 2] = 0.10000000;
    bw.trans_prob[1*N + 0] = 0.30000000;
    bw.trans_prob[1*N + 1] = 0.30000000;
    bw.trans_prob[1*N + 2] = 0.40000000;
    bw.trans_prob[2*N + 0] = 0.10000000;
    bw.trans_prob[2*N + 1] = 0.10000000;
    bw.trans_prob

In [5]:
# test_case_ghmm_3

A = np.array([[0.5, 0.4, 0.1], [0.3, 0.3, 0.4], [0.1, 0.1, 0.8]], dtype=np.float64)
B = np.array([[0.1, 0.0, 0.9], [0.05, 0.95, 0.0], [0.3, 0.52, 0.18]], dtype=np.float64)
pi = np.array([0.1, 0.5, 0.4], dtype=np.float64)
Y = np.array([2, 1, 0, 1, 0, 2, 2, 2, 1, 1, 2, 2, 0, 1, 1, 0], np.int)

nrSteps = 1000
loglikelihoodCutoff = 0
sigma = IntegerRange(0,3)

print_observations(Y, "bw")
print_model(A, B, pi, "bw")
hmm_model = HMMFromMatrices(sigma, DiscreteDistribution(sigma), list(A), list(B), list(pi))
train_seq = EmissionSequence(sigma, list(Y))
hmm_model.baumWelch(train_seq, nrSteps, loglikelihoodCutoff)
print_converged_model(hmm_model, A, B, pi, "bw_check")



GHMM ghmm.py:148 - reestimate.c:ghmm_dmodel_baum_welch(843): No convergence: log P < log P-old! (n=93)


    bw.observations[0*T + 0] = 2;
    bw.observations[0*T + 1] = 1;
    bw.observations[0*T + 2] = 0;
    bw.observations[0*T + 3] = 1;
    bw.observations[0*T + 4] = 0;
    bw.observations[0*T + 5] = 2;
    bw.observations[0*T + 6] = 2;
    bw.observations[0*T + 7] = 2;
    bw.observations[0*T + 8] = 1;
    bw.observations[0*T + 9] = 1;
    bw.observations[0*T + 10] = 2;
    bw.observations[0*T + 11] = 2;
    bw.observations[0*T + 12] = 0;
    bw.observations[0*T + 13] = 1;
    bw.observations[0*T + 14] = 1;
    bw.observations[0*T + 15] = 0;

    bw.init_prob[0] = 0.10000000;
    bw.init_prob[1] = 0.50000000;
    bw.init_prob[2] = 0.40000000;

    bw.trans_prob[0*N + 0] = 0.50000000;
    bw.trans_prob[0*N + 1] = 0.40000000;
    bw.trans_prob[0*N + 2] = 0.10000000;
    bw.trans_prob[1*N + 0] = 0.30000000;
    bw.trans_prob[1*N + 1] = 0.30000000;
    bw.trans_prob[1*N + 2] = 0.40000